In [8]:
import requests as rq
import pandas as pd
import numpy as np
import re

In [9]:
base_urls = rq.get('https://swapi.dev/api/').json()
base_urls
def getAllResponse(url):
    data_response = rq.get(url).json()
    data = []
    data+=data_response['results']

    while(data_response['next'] != None):
        data_response = rq.get(data_response['next']).json()
        data+=data_response['results']
    data = pd.DataFrame(data)   
    return data
    

In [10]:
def cleanColumns(columns, data):
    for column in columns:
        data[column] = data[column].map(lambda x :  [re.findall(r'\d+', url)[0] for url in x])

In [11]:
base_urls

{'people': 'https://swapi.dev/api/people/',
 'planets': 'https://swapi.dev/api/planets/',
 'films': 'https://swapi.dev/api/films/',
 'species': 'https://swapi.dev/api/species/',
 'vehicles': 'https://swapi.dev/api/vehicles/',
 'starships': 'https://swapi.dev/api/starships/'}

In [12]:
people = getAllResponse(base_urls['people'])
films = getAllResponse(base_urls['films'])
vehicles = getAllResponse(base_urls['vehicles'])
starships = getAllResponse(base_urls['starships'])
species = getAllResponse(base_urls['species'])
planets = getAllResponse(base_urls['planets'])



In [13]:

people_columns_to_clean = ['films', 'species', 'vehicles', 'starships']
cleanColumns(people_columns_to_clean, people)
people['homeworld'] = people['homeworld'].map(lambda x :  re.findall(r'\d+', str(x))[0] if len( re.findall(r'\d+', str(x))) > 0 else None)
people['url'] = people['url'].map(lambda x :  re.findall(r'\d+', x)[0])
del people['created']
del people['edited']



In [14]:
people.rename({"url" :"id"},axis=1, inplace=True)
films.rename({"url" :"id"},axis=1, inplace=True)
starships.rename({"url" :"id"},axis=1, inplace=True)
vehicles.rename({"url" :"id"},axis=1, inplace=True)
planets.rename({"url" :"id"},axis=1, inplace=True)
species.rename({"url" :"id"},axis=1, inplace=True)


In [15]:
people['species'] = people.species.apply(lambda x: None if len(x)==0 else x[0])
people.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,id
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,1,"[1, 2, 3, 6]",None,"[14, 30]","[12, 22]",1
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,1,"[1, 2, 3, 4, 5, 6]",2,[],[],2
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,8,"[1, 2, 3, 4, 5, 6]",2,[],[],3
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,1,"[1, 2, 3, 6]",None,[],[13],4
4,Leia Organa,150,49,brown,light,brown,19BBY,female,2,"[1, 2, 3, 6]",None,[30],[],5


In [16]:
pilot_starships = people.explode('starships')[['id','starships']].dropna()
pilot_starships.columns = ['character_id', 'starship_id']

vehicles_drivers = people.explode('vehicles')[['id', 'vehicles']].dropna()
vehicles_drivers.columns = ['character_id', 'vehicle_id']

characters_in_films = people.explode('films')[['id', 'films']]
characters_in_films.columns = ['character_id', 'film_id']

del people['vehicles']
del people['starships']
del people['films']

In [17]:
#people['species'] = people.species.apply(lambda x: None if len(x)==0 else x[0])
people = people.replace(to_replace='n/a',value=None)
people = people.replace(to_replace='N/A',value=None)
people.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,id
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,1,None,1
1,C-3PO,167,75,None,gold,yellow,112BBY,None,1,2,2
2,R2-D2,96,32,None,"white, blue",red,33BBY,None,8,2,3
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,1,None,4
4,Leia Organa,150,49,brown,light,brown,19BBY,female,2,None,5


In [18]:
films.head()

,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,id
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/4/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/4/, https://swa...","[https://swapi.dev/api/starships/3/, https://s...","[https://swapi.dev/api/vehicles/8/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-12T11:26:24.656000Z,2014-12-15T13:07:53.386000Z,https://swapi.dev/api/films/2/
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/2/, https://s...","[https://swapi.dev/api/vehicles/8/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-18T10:39:33.255000Z,2014-12-20T09:48:37.462000Z,https://swapi.dev/api/films/3/
3,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/31/, https://...","[https://swapi.dev/api/vehicles/33/, https://s...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-19T16:52:55.740000Z,2014-12-20T10:54:07.216000Z,https://swapi.dev/api/films/4/
4,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/planets/1/, https://swa...","[https://swapi.dev/api/starships/21/, https://...","[https://swapi.dev/api/vehicles/4/, https://sw...","[https://swapi.dev/api/species/1/, https://swa...",2014-12-20T10:57:57.886000Z,2014-12-20T20:18:48.516000Z,https://swapi.dev/api/films/5/


In [19]:
films_columns_to_clean = ['planets','starships','vehicles']
cleanColumns(films_columns_to_clean, films)
films['id'] = films['id'].map(lambda x :  re.findall(r'\d+', x)[0])
del films['created']
del films['edited']
del films['characters']
del films['species']

films.head()

,title,episode_id,opening_crawl,director,producer,release_date,planets,starships,vehicles,id
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]",1
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[4, 5, 6, 27]","[3, 10, 11, 12, 15, 17, 21, 22, 23]","[8, 14, 16, 18, 19, 20]",2
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"[1, 5, 7, 8, 9]","[2, 3, 10, 11, 12, 15, 17, 22, 23, 27, 28, 29]","[8, 16, 18, 19, 24, 25, 26, 30]",3
3,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"[1, 8, 9]","[31, 32, 39, 40, 41]","[33, 34, 35, 36, 37, 38, 42]",4
4,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"[1, 8, 9, 10, 11]","[21, 32, 39, 43, 47, 48, 49, 52, 58]","[4, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57]",5


In [20]:
planets_in_films = films.explode('planets')[['id', 'planets']]
planets_in_films.columns = ['film_id', 'planet_id']

vehicles_in_films = films.explode('vehicles')[['id','vehicles']]
vehicles_in_films.columns = ['film_id', 'vehicle_id']



starships_in_films = films.explode('starships')[['id','starships']]
starships_in_films.columns = ['film_id', 'starship_id']

films_producers = pd.DataFrame(films.producer.str.split(',').tolist(), index=films.id).stack()
films_producers = films_producers.reset_index()[[0, 'id']] 
films_producers.columns = ['producer', 'film_id'] 
films_producers

del films['planets']
del films['vehicles']
del films['starships']
del films['producer']

In [21]:

films.head()

,title,episode_id,opening_crawl,director,release_date,id
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,1977-05-25,1
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,1980-05-17,2
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,1983-05-25,3
3,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,1999-05-19,4
4,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,2002-05-16,5


In [22]:

vehicles['id'] = vehicles['id'].map(lambda x :  re.findall(r'\d+', x)[0])
del vehicles['created']
del vehicles['edited']
del vehicles['pilots']
del vehicles['films']
vehicles.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,id
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,4
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,6
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,7
3,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,8
4,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,14


In [23]:

starships['id'] = starships['id'].map(lambda x :  re.findall(r'\d+', x)[0])
del starships['created']
del starships['edited']
del starships['pilots']
del starships['films']

starships.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,id
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,2
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,3
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,5
3,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,9
4,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,10


In [24]:

species['homeworld'] = species['homeworld'].map(lambda x :  re.findall(r'\d+', str(x))[0] if len( re.findall(r'\d+', str(x))) > 0 else None)
species['id'] = species['id'].map(lambda x :  re.findall(r'\d+', x)[0])
del species['created']
del species['edited']
del species['people']
del species['films']
species.head()

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,id
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,9,Galactic Basic,1
1,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,None,n/a,2
2,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,14,Shyriiwook,3
3,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,23,Galatic Basic,4
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,24,Huttese,5


In [25]:

planets['id'] = planets['id'].map(lambda x :  re.findall(r'\d+', x)[0])
del planets['created']
del planets['edited']
del planets['residents']
del planets['films']
planets.head()

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,id
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,1
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,2
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,3
3,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,4
4,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,5


In [27]:

starships = starships.replace(to_replace='n/a',value=None)
planets = planets.replace(to_replace='n/a',value=None)
vehicles = vehicles.replace(to_replace='n/a',value=None)
species = species.replace(to_replace='n/a',value=None)
starships = starships.replace(to_replace='N/A',value=None)
planets = planets.replace(to_replace='N/A',value=None)
vehicles = vehicles.replace(to_replace='N/A',value=None)
species = species.replace(to_replace='N/A',value=None)

In [28]:
starships['cost_in_credits'] = pd.to_numeric(starships['cost_in_credits'], errors='coerce')

In [ ]:
people['height'] = people['height'].apply(lambda x: int(x) if x.isdigit() else 0)
people['mass'] = people['mass'].apply(lambda x: int(x) if x.isdigit() else 0)

<h1> SQL </h1>

In [30]:
import sqlite3

<h3> Converting all datatypes of dataframes to respective correct datatypes </h3>

In [41]:
convert_dict = {	
    'title' : str ,
	'episode_id' : int,
	'opening_crawl' : str,
	'director' : str,
	'id' : int
                }
 
films = films.astype(convert_dict)
films['release_date'] = pd.to_datetime(films['release_date'])

In [42]:
convert_dict = {	
	'planet_id' : int,
	'film_id' : int
                }
 
planets_in_films = planets_in_films.astype(convert_dict)

In [43]:
convert_dict = {	
	'vehicle_id' : int,
	'film_id' : int
                }
 
vehicles_in_films = vehicles_in_films.astype(convert_dict)

In [44]:
convert_dict = {	
	'starship_id' : int,
	'film_id' : int
                }
 
starships_in_films = starships_in_films.astype(convert_dict)

In [45]:
convert_dict = {	
	'film_id' : int,
	'producer' : str
                }
 
films_producers = films_producers.astype(convert_dict)

In [48]:
convert_dict = {	
    'name' : str ,
	'birth_year' : str,
	'eye_color' : str,
	'gender' : str,
	'hair_color' : str,
	'height' : float,
	'mass' : float,
	'skin_color' : str,
	'id' : int
                }
 
people = people.astype(convert_dict)
people['species'] = people['species'].fillna(0).astype(int)

In [50]:
convert_dict = {	
	'character_id' : int,
	'film_id' : int
                }
 
characters_in_films = characters_in_films.astype(convert_dict)

In [ ]:
pilot_starships = {	
	'character_id' : int,
	'starship_id' : int
                }
 
pilot_starships = pilot_starships.astype(convert_dict)

In [ ]:
convert_dict = {	
	'character_id' : int,
	'vehicle_id' : int
                }
 
vehicles_drivers = vehicles_drivers.astype(convert_dict)

In [51]:
# The rest of the dataframes only have one column 'id' that needs to be set to int

planets['id'] = planets['id'].astype(int)
species['id'] = species['id'].astype(int)
starships['id'] = starships['id'].astype(int)
vehicles['id'] = vehicles['id'].astype(int)

<h3> Creating Tables and Inserting Data </h3>

In [58]:

conn = sqlite3.connect('starwars_Database.db')
c = conn.cursor()

c.execute(f"CREATE TABLE IF NOT EXISTS films ({', '.join(films.columns)}, PRIMARY KEY (id))")
conn.commit()
films.to_sql('films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS planets_in_films ({', '.join(planets_in_films.columns)})")
conn.commit()
planets_in_films.to_sql('planets_in_films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles_in_films ({', '.join(vehicles_in_films.columns)})")
conn.commit()
vehicles_in_films.to_sql('vehicles_in_films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS starships_in_films ({', '.join(starships_in_films.columns)})")
conn.commit()
starships_in_films.to_sql('starships_in_films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS films_producers ({', '.join(films_producers.columns)})")
conn.commit()
films_producers.to_sql('films_producers', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS people ({', '.join(people.columns)} , PRIMARY KEY (id))")
conn.commit()
people.to_sql('people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS characters_in_films ({', '.join(characters_in_films.columns)})")
conn.commit()
characters_in_films.to_sql('characters_in_films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS pilot_starships ({', '.join(pilot_starships.columns)})")
conn.commit()
pilot_starships.to_sql('pilot_starships', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles_drivers ({', '.join(vehicles_drivers.columns)})")
conn.commit()
vehicles_drivers.to_sql('vehicles_drivers', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS planets ({', '.join(planets.columns)} , PRIMARY KEY (id))")
conn.commit()
planets.to_sql('planets', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS species ({', '.join(species.columns)} , PRIMARY KEY (id))")
conn.commit()
species.to_sql('species', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS starships ({', '.join(starships.columns)} , PRIMARY KEY (id))")
conn.commit()
starships.to_sql('starships', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles ({', '.join(vehicles.columns)} , PRIMARY KEY (id))")
conn.commit()
vehicles.to_sql('vehicles', conn, if_exists='replace', index = False)


#  Clement, if you could please comment on this issue I am facing, When i define Primary Key inside the brackets for table creation,
#  it doesnt give out any errors or anything, but I dont think it marks it as the actual primary key. This has never happened to me 
#  while working with Transact SQL in Microsoft SQL Server Studio

# I know it doesnt matter as long as you know what your foreign key constraints are while writing queries. But it would be cool to know 
# where I am screwing up with Primary Keys in SQLlite.

39

<h1> Queries from SQL </h1>

<h4>Get all the starships that were manufactured by Corellian Engineering Corporation and were featured in the movies released between 1980 and 2000 and </h4>


In [84]:
con = sqlite3.connect('starwars_Database.db')
pd.read_sql(
    '''
        SELECT f.title, s.name FROM
            starships_in_films as sf
        INNER JOIN starships as s on sf.starship_id = s.id
        INNER JOIN films as f on sf.film_id = f.id
        WHERE 
            (f.release_date BETWEEN '1980-01-01' AND '2000-01-01') 
            AND 
            (s.manufacturer = 'Corellian Engineering Corporation')
        ORDER BY
            f.release_date  
    ''', 
    con
)

,title,name
0,The Empire Strikes Back,Millennium Falcon
1,Return of the Jedi,CR90 corvette
2,Return of the Jedi,Millennium Falcon
3,The Phantom Menace,Republic Cruiser


<h4>Get the charaters that appeared in all movies</h4>

In [95]:
con = sqlite3.connect('starwars_Database.db')
pd.read_sql(
    '''
        SELECT 
            p.name
        FROM
            people as p
        LEFT JOIN 
            characters_in_films cf on cf.character_id = p.id
        LEFT JOIN 
            films f on cf.film_id = f.id
        GROUP BY
            p.name
        HAVING 
            COUNT(1) = (SELECT COUNT(1) FROM films)
    ''', 
    con
)

,name
0,C-3PO
1,Obi-Wan Kenobi
2,R2-D2


<h4>Get number of vehicles driven by characters </h4>

In [100]:
con = sqlite3.connect('starwars_Database.db')
pd.read_sql(
    '''
        SELECT 
            p.name,
            count(*) as number_of_vehicles
        FROM 
            vehicles_drivers as vd
        INNER JOIN 
            people as p on p.id = vd.character_id
        INNER JOIN 
            vehicles as v on v.id = vd.vehicle_id
        GROUP BY
            vd.character_id
        ORDER BY number_of_vehicles DESC
    ''', 
    con
)

,name,number_of_vehicles
0,Anakin Skywalker,2
1,Luke Skywalker,2
2,Grievous,1
3,Zam Wesell,1
4,Dooku,1
5,Leia Organa,1
6,Darth Maul,1
7,Qui-Gon Jinn,1
8,Wedge Antilles,1
9,Chewbacca,1


<h4>Making a CTE that contains all information about a character and its vehicle and starship driven in movies</h4>
<p>Filtering the first released movie and checking which character both drove and piloted which vehicle and starship</p>

In [119]:
con = sqlite3.connect('starwars_Database.db')
pd.read_sql(
    '''
        WITH cte_drivers_pilots AS (
            SELECT 
                p.name as character_name,
                planets.name as homeworld,
                f.title as movie,
                f.release_date,
                s.name as starship_name,
                s.max_atmosphering_speed as starship_max_atmosphering_speed,
                s.cost_in_credits as starship_cost_in_credits,
                s.manufacturer as starship_manufacturer,
                v.name as vehicle_name,
                v.max_atmosphering_speed as vehicle_max_atmosphering_speed,
                v.cost_in_credits as vehicle_cost_in_credits,
                v.manufacturer as vehicle_manufacturer

            FROM 
                people as p
            INNER JOIN 
                characters_in_films as cf on p.id = cf.character_id
            LEFT JOIN 
                pilot_starships as ps on p.id = ps.character_id
            INNER JOIN
                starships_in_films as sif on sif.starship_id = ps.starship_id
            LEFT JOIN 
                vehicles_drivers as vd on p.id = vd.character_id
            INNER JOIN
                vehicles_in_films as vif on vif.vehicle_id = vd.vehicle_id
            INNER JOIN
                films as f on f.id = sif.film_id or f.id = vif.film_id
            LEFT JOIN
                starships as s on s.id = ps.starship_id
            LEFT JOIN
                vehicles as v on v.id = vd.vehicle_id
            LEFT JOIN 
                planets on planets.id = p.homeworld
            GROUP BY 
                p.id
        )
        SELECT character_name, starship_name, vehicle_name, movie
        from cte_drivers_pilots
        WHERE movie = (SELECT title FROM films ORDER BY release_date LIMIT 1)
        
    ''', 
    con
)

,character_name,starship_name,vehicle_name,movie
0,Luke Skywalker,X-wing,Snowspeeder,A New Hope
1,Chewbacca,Millennium Falcon,AT-ST,A New Hope
2,Wedge Antilles,X-wing,Snowspeeder,A New Hope
